In [718]:
#When updating the imported Vivado files (.bit, .hwh, .tcl), it is necessary to restart the PL.
from pynq import PL
PL.reset()

In [729]:
#Libraries and Overlay upload

from pynq import Overlay, Clocks, allocate
from pynq import allocate
import numpy as np
import matplotlib.pyplot as plt
import time

overlay = Overlay('trig_uart1msps.bit')

print("Accesible IP-Cores:")
overlay.ip_dict.keys()

Accesible IP-Cores:


dict_keys(['axi_dma_0', 'GPIO_CTRL/DECIM_AND_SIGNAL_SEL', 'GPIO_CTRL/THRESHOLD_CTRL', 'GPIO_CTRL/PRETRIG_AND_WINDOW_CTRL', 'GPIO_CTRL/TEST_SIGNALS', 'processing_system7_0'])

In [739]:
#DMA initialization
dma = overlay.axi_dma_0   
dma_recv = dma.recvchannel

GPIO = overlay.GPIO_CTRL

#--------------------------------------------------------------------

#GPIO control of pretrigger samples and total samples in window
xgpio_pretrig_wind = GPIO.PRETRIG_AND_WINDOW_CTRL


xgpio_pretrig_wind.write(0,1024) #Total window samples
xgpio_pretrig_wind.write(8,100)  #Pretrigger samples

#--------------------------------------------------------------------

#GPIO control of decimation factor and reference signal selection
xgpio_decim_signal_sel = GPIO.DECIM_AND_SIGNAL_SEL

decim_factor = 1

xgpio_decim_signal_sel.write(0, decim_factor) #Decimation factor (1 for decimation off)

#Predefined reference signal selection
#0x0 DC CONSTANT 0.5v monopolar mode
#0x1 FILTERED SIGNAL
#0x2 NOISY SIGNAL
#0x3 EXPONENTIAL
xgpio_decim_signal_sel.write(8, 0x2) #signal selection

#--------------------------------------------------------------------

#GPIO control of threshold_control
xgpio_thresh = GPIO.THRESHOLD_CTRL

thresh_level = 50000
xgpio_thresh.write(0, thresh_level)  #Threshold Level
xgpio_thresh.write(8, 0x0) #Edge sel

#--------------------------------------------------------------------

#GPIO control of XADC TESTING
xgpio_reference = GPIO.TEST_SIGNALS

In [740]:
xgpio_reference.read(0)  #Reference value (XADC TESTING)

19497

En este caso se hace el uso del XADC de la PYNQ en modo unipolar para voltajes de referencia de 0V a 1V, en el caso de configurarse mediante modificaciones en el diseño de hardaware en modo bipolar, se tienen valores de referencia de -0.5V a 0.5V

In [741]:
buffer_size = 1040
#DMA buffer size assignment

#Input buffer declaration
input_buffer = allocate(shape=(buffer_size,)) #, dtype=np.uint16)
data_resol=16

In [ ]:
data_out = []
t, t_0 = [], 0
cycles = 1
start = time.time()

XADC_SR = 1000000 #XADC Sample Rate in samples/sec

for i in range(0,cycles):
    
    #time.sleep(.1)
    dma.recvchannel.transfer(input_buffer)
    dma.recvchannel.wait()
    
    data_out.append(input_buffer/2**16)  
    
    
data_out = np.array(data_out).flatten()

print('ADQUISITION READY')
print('DMA idle Status:',dma.recvchannel.idle)

#Time sampling
t_factor = len(data_out)*decim_factor/XADC_SR*1e6 #us scale

t = np.linspace(0, t_factor, len(data_out))


plt.figure(figsize=(15, 4))
plt.plot(t, data_out)
#plt.hlines(thresh_level, 0, buffer_size, linestyle = 'dashed', color = 'r', alpha=.5)

#Subdivision of individual DMA adquisition
#for k in range(0, cycles):
#    plt.vlines(1024*k, 0, 2**data_resol, color='red', linestyle='dashed')
    
plt.title("DMA read")

plt.xlabel("A.U.")
plt.ylabel("Dig. value")
#plt.xlim(0, 100)
plt.grid()

plt.show()

#np.savetxt("data_out.txt", data_out)

In [708]:
del input_buffer

In [32]:
#Address 28 for XADC input analog pins VAUXP12=A6, VAUXN12=A7
XADC_SR = 1000000 #XADC Sample Rate in samples/sec

#DMA initialization
#dma = overlay.axi_dma_0   
#dma_recv = dma.recvchannel

data_out = []
t, t_0 = [], 0
cycles = 1
start = time.time()


for i in range(0,cycles):
    #time.sleep(.1)
    
    #Transfer FIFO memory to input_buffer
    #if dma.recvchannel.idle:
    dma.recvchannel.transfer(input_buffer)
    dma.recvchannel.wait()
    
    #Data normalization to voltage limits reference 0v -1v (Unipolar Mode XADC)
    input_buffer1 = input_buffer/2**data_resol
    
    data_out.append(input_buffer1[:buffer_size-4])  
    
    #t.append(np.linspace(0, t_factor, len(input_buffer1[:16384])))
    

#Implementar en caso de tener XADC modo Bipolar(-0.5V --> 0.5V)

#for i in range(len(input_buffer1)):
#    if  input_buffer1[i]>0.5:
#        input_buffer1[i] = input_buffer1[i]-1

data_out = np.array(data_out).flatten()


#Time sampling
t_factor = len(input_buffer1[:buffer_size-4])/XADC_SR * 1000

t = np.linspace(0, t_factor*cycles, len(input_buffer1[:buffer_size-4])*cycles)

print('ADQUISITION READY')


print('DMA idle Status:',dma.recvchannel.idle)

plt.figure(figsize=(15, 4))
plt.plot(t, data_out)

#Subdivision of individual DMA adquisition
for k in range(0, cycles):
    plt.vlines(t_factor*k, 0, 1, color='red', linestyle='dashed')
    
plt.title("XADC Samples")
plt.xlabel("time (t) in ms")
plt.ylabel("XADC Monopolar Volt. Level")
#plt.ylim(200, 250)
plt.grid()
plt.show()
print(f"XADC Sampling Rate: {XADC_SR} SPS \n{len(data_out)} samples in {t[-1]} ms")
#np.savetxt("data_out.txt", data_out)


KeyboardInterrupt: 